Tracking Objects in Video with Particle Filters
===============================================

Import libraries

In [1]:
import numpy as np
import cv2

# Repeatability
np.random.seed(0)

VFILENAME = "walking.mp4"
HEIGHT = 406
WIDTH = 722

Load video frames from file

In [2]:
def get_frames(filename):
  video=cv2.VideoCapture(filename)
  
  while video.isOpened():
    # returning a flag and frame 
    ret,frame=video.read()
    if ret:
      yield frame
    else:
      break
  
  # Releasing the resources
  video.release()
  yield None

Creating a particle cloud

In [ ]:
def initialize_particles():
    return []

Moving particles according to their velocity state

In [ ]:
def apply_velocity(particles):
    return particles

Prevent particles from falling off the edge of the video frame

In [ ]:
def enforce_edges(particles):
    return particles

Measure each particle's quality

In [ ]:
def compute_errors(particles, frame):
    return []

Assign weights to the particles based on their quality of match

In [ ]:
def compute_weights(errors):
    return []

Resample particles according to their weights

In [ ]:
def resample(particles, weights):
    return particles, []

Fuzz the particles

In [ ]:
def apply_noise(particles):
    return particles

Display the video frames

In [ ]:
def display(frame, particles, location):
    return False

Main routine

In [ ]:
particles = initialize_particles()

for frame in get_frames(VFILENAME):
    if frame is None: break

    particles = apply_velocity(particles)
    particles = enforce_edges(particles)
    errors = compute_errors(particles, frame)
    weights = compute_weights(errors)
    particles, location = resample(particles, weights)
    particles = apply_noise(particles)
    terminate = display(frame, particles, location)
    if terminate:
        break
cv2.destroyAllWindows()
